# Word classification with a Character-level RNN

이번 실습에서는 순환 신경망(RNN, Recurrent Neural Network)을 이용하여 단어를 분류하는 모델을 학습해 보겠습니다

구체적으로는 18개 언어에서 수집된 성(姓, surname) 데이터를 활용합니다. RNN 모델은 이름을 구성하는 문자(character)들을 하나씩 입력받아, 최종적으로 "이 이름이 어떤 언어에서 왔는지?"를 예측하게 됩니다.

In [ ]:
import string
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from helpers import draw_confusion_matrix, train_one_epoch, evaluate_one_epoch

데이터를 간단히 살펴보겠습니다.

- `name`열에는 실제 사람들의 성(surname)이 들어 있습니다
- `language`열에는 각 이름이 어느 언어에서 유래했는지를 나타내는 라벨(label)이 들어 있습니다.

In [ ]:
df = pd.read_csv('/datasets/NLP/names.csv')

print(df.sample(n = 10))

총 18개 언어 카테고리가 있으며, 

가장 많은 데이터를 가진 클래스는 Russian (9,408개), 반대로 가장 적은 클래스는 Vietnamese (73개) 입니다.

- 데이터 불균형(data imbalance)이 상당히 심각하며, 이는 학습 과정에서 모델이 데이터가 많은 클래스에 치우칠 수 있음을 의미합니다.
- 예를 들어, 모델은 러시아어 성을 쉽게 잘 맞출 수 있지만, 베트남어나 포르투갈어 성은 제대로 학습하지 못하여 예측 성능이 떨어질 수 있습니다.

In [ ]:
class_names = sorted(df['language'].unique().tolist())
print("target categories:", class_names, "\n")

for class_name in class_names:
    print(f"Class counts for '{class_name}': {len(df[df['language'] == class_name])}")

## 문자 단위 토큰화 (Character-level tokenization)
자연어 처리에서 텍스트를 신경망에 입력하기 위해서는 먼저 토큰화(tokenization)과정을 거쳐야 합니다.
- 토큰(token)이란 텍스트를 처리하기 위해 정의한 가장 작은 처리 단위를 의미합니다
- 문자 단위 토큰화에서는 문자(character)를 기본 단위로 토큰화를 수행합니다

예를들어 문자열 "Kim"을 문자 단위로 토큰화하면 다음과 같은 시퀀스로 변환됩니다

$$
\text{Kim} \;\;\longrightarrow\;\; [\text{K}, \text{i}, \text{m}]
$$

## Vocalulary
Vocabulary $V$는 가능한 모든 토큰들의 집합을 의미합니다
- 문자 단위 토큰화에서는 알파벳, 숫자, 공백, 특수문자 등이 Vocabulary를 이룹니다.
- $V=${'a', 'b', 'c', ..., 'Y', 'Z', ' ', '.', ','}

In [ ]:
VOCAB = list(string.ascii_letters + " .,;'" + "_")
VOCAB_SIZE = len(VOCAB)
INDEX_TO_CHAR = {i: c for i, c in enumerate(VOCAB)}
CHAR_TO_INDEX = {c: i for i, c in enumerate(VOCAB)}

print("Vocabulary:", VOCAB)
print("Vocabulary size:", VOCAB_SIZE)
print("Index to char:", INDEX_TO_CHAR)
print("Char to index:", CHAR_TO_INDEX)

## One-hot Encoding 
텍스트 데이터를 신경망에 입력하기 위해서는 각 문자(토큰)를 수치적 표현(`Tensor`)으로 변환해야 합니다. 이때, 가장 기본적으로 사용할 수 있는 방법이 One-hot encoding 입니다.

- 하나의 문자를 one-hot encoding으로 표현한다면 $\{0, 1\}^{|V|}$ 형태의 벡터로 표현됩니다.
  - 여기서 |V|는 Vocabulary 크기(VOCAB_SIZE)를 의미합니다.
  - One-hot encoded 텐서는 해당 문자의 인덱스 위치에만 1의 값을 가지고 나머지 모든 위치에는 0의 값을 가집니다.
  - 예: "b" → `[0 1 0 0 0 ...]`
- 하나의 문자를 one-hot encoding 하면 $(1, |V|)$의 shape을 가진 텐서가 됩니다.
- 문자열 전체를 인코딩하면 $(\text{len(text)}, |V|)$의 shape을 가진 텐서가 됩니다.

<mark>실습</mark> one_hot_encode_string함수를 완성하세요.
- 이 함수는 문자열(string)을 입력받아 각 문자에 대해 one-hot encoding을 수행한 뒤, 문자열을 표현하는 텐서를 리턴합니다
- `CHAR_TO_INDEX`를 활용하세요.


In [ ]:
def one_hot_encode_string(text):
    """
    Convert a string into a tensor of shape [len(text), |V|].
    """
    encoded_tensor = ...  # TODO: Initialze a zero tensor using torch.zeros
    for t, letter in enumerate(text):
        ##### YOUR CODE START #####
        
        ##### YOUR CODE END #####
    return encoded_tensor

In [ ]:
print (f"The letter 'a' becomes: \n{one_hot_encode_string('a')}")
print (f"The name 'Ahn' becomes: \n{one_hot_encode_string('Ahn')}")

**Expected output:**

```python
The letter 'a' becomes: 
tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])
The name 'Ahn' becomes: 
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])
```

## Dataset
PyTorch의 `Dataset`객체는 한번에 하나의 데이터와 라벨을 읽어오는 기능을 제공합니다.

- `len(dataset)` : 전체 데이터 개수를 반환합니다.
- `dataset[i]` : 인덱스 $i$를 전달하면 $i$번째 샘플(example)의 데이터와 라벨 `(X, y)` 을 반환합니다.

<b>사용자 정의 데이터셋 (Custom dataset)</b>을 정의하려면 다음 세 가지 메서드를 override 해야 합니다:

- `__init__` : 생성자
- `__len__`을 구현하여 `len(dataset)` 호출 시 데이터셋의 크기 (전체 샘플 수)를 반환하도록 합니다.
- `__getitem__` 을 구현하여 `dataset[i]` 호출 시 `i`번째 샘플의 데이터와 라벨 `(X, y)`을 반환하도록 합니다.


<mark>실습</mark> `NamesDataset`을 완성하세요
- 이름(`name_str`)은 `one_hot_encode_string`함수를 이용하여 텐서로 변환합니다.
- 라벨(`label_str`)은 `self.class_names` 리스트에서 해당 클래스의 인덱스를 찾아 정수형 라벨(`label_index`)로 변환합니다.
  - `list.index()` 메서드를 활용하세요

In [ ]:
class NamesDataset(Dataset):
    """ Custom Dataset for name classification."""
    def __init__(self, csv_path):
        super().__init__()
        self.df  = pd.read_csv(csv_path)
        self.class_names = sorted(self.df['language'].unique().tolist())
        self.num_classes = len(self.class_names)
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        name_str, label_str = self.df[["name", "language"]].iloc[idx]
        
        name_tensor = ...  # TODO
        label_index = ...  # TODO

        label_tensor = torch.tensor(label_index, dtype=torch.long)
        return name_tensor, label_tensor

    def get_sample_weights(self):
        """
        Compute weights for each sample to balance the dataset.
        
        Returns:
            List[float]: Weight for each sample (inverse of label frequency).
        """
        class_counts = [len(self.df[self.df['language'] == class_name]) for class_name in self.class_names]
        class_weights = [1.0 / count for count in class_counts]
        sample_weights = [class_weights[self.class_names.index(class_name)] for class_name in self.df["language"]]
        return sample_weights
    

In [ ]:
names_dataset = NamesDataset(csv_path='/datasets/NLP/names.csv')

X, y = names_dataset[0]
print("X.shape:", X.shape)
print("y:", y)

In [ ]:
names_dataloader = DataLoader(names_dataset, batch_size = 1, shuffle=True)
for X_batch, y_batch in names_dataloader:
    print("X_batch.shape:", X_batch.shape)
    print("y_batch:", y_batch)
    break

### WeightedRandomSampler

앞서 확인하였던 클래스 불균형 (class imbalance) 문제를 해결하기 위한 방법 중 하나로 `WeightedRandomSampler`를 사용해보겠습니다.

- 이 샘플러는 `DataLoader`가 미니배치(mini-batch)를 구성할 때, 각 샘플에 부여된 가중치에 따라 선택 확률을 조정합니다.
  - 즉, 희귀한 클래스(샘플 수가 적은 클래스)에 속하는 데이터는 더 자주 선택되도록 하고, 샘플 수가 많은 클래스에 속하는 데이터는 상대적으로 덜 선택되도록 할 수 있습니다.
- `WeightedRandomSampler`를 각 샘플별 가중치 `sample_weights`에 비례하여 샘플링하도록 합니다. 이때 `sample_weights`는 다음과 같이 계산됩니다
  - 각 클래스의 등장 빈도를 계산합니다 (`class_counts`).
  - 각 클래스의 가중치를 1 / (해당 클래스의 샘플 수)로 설정합니다 (`class_weights`). 즉, 샘플 수가 적을 수록 가중치가 커집니다.
  - 각 샘플에는 자신이 속한 클래스의 가중치가 할당됩니다 (`sample_weights`).
- 데이터가 부족한 클래스는 oversampling되어 한 샘플이 여러 번 중복하여 재사용될 수 있습니다.
- 데이터가 충분한 클래스는 undersampling되어 전체 데이터 중 일부만이 사용될 수 있습니다.

In [ ]:
sample_weights = names_dataset.get_sample_weights()
sampler = torch.utils.data.sampler.WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

dataloader_with_sampler = DataLoader(names_dataset, batch_size = 1, sampler=sampler)
dataloader_without_sampler = DataLoader(names_dataset, batch_size = 1, shuffle=True)


counter_without_sampler = [0] * names_dataset.num_classes
for X_batch, y_batch in dataloader_without_sampler:
    counter_without_sampler[y_batch.item()] += 1

print("Class distribution without Sampler:\n", counter_without_sampler)

counter_with_sampler = [0] * names_dataset.num_classes
for X_batch, y_batch in dataloader_with_sampler:
    counter_with_sampler[y_batch.item()] += 1

print("Class distribution with WeightedRandomSampler:\n", counter_with_sampler)

## Recurrent Neural Network (RNN)

RNN은 sequential data를 처리하기 위한 기본적인 신경망 구조 중 하나입니다.

일반적인 신경망은 입력 전체를 한 번에 처리하는 반면, RNN은 시간 축을 따라 입력데이터를 한번에 하나씩 <b>순차적으로 처리</b>합니다. 이 과정에서 hidden state $\mathbf{h}_t$는 이전 시점의 정보를 요약한 메모리(memory)역할을 수행하므로, 과거의 맥락을 현재 시점의 계산에 반영할 수 있습니다.

- **Hidden state update**:

$$
\mathbf{h}_t = \tanh(\mathbf{W}_{xh}\mathbf{x}_t + \mathbf{W}_{hh}\mathbf{h}_{t-1} + b_h),\quad t=1,...,T
$$

- **Output generation**:

$$
\mathbf{y}_T = \mathbf{W}_{hy}\mathbf{h}_T + b_y
$$

Where:
- $\mathbf{x}_t \in \mathbb{R}^{d_x}$ is the input vector at time step $t$
- $\mathbf{h}_t \in \mathbb{R}^{d_h}$ is the hidden state vector at time step $t$
- $\mathbf{y}_T \in \mathbb{R}^{d_y}$ is the output vector at final time step $T$
- $\mathbf{W}_{xh} \in \mathbb{R}^{d_h \times d_x}$ is the input-to-hidden weight matrix
- $\mathbf{W}_{hh} \in \mathbb{R}^{d_h \times d_h}$ is the hidden-to-hidden weight matrix
- $\mathbf{W}_{hy} \in \mathbb{R}^{d_y \times d_h}$ is the hidden-to-output weight matrix
- $b_h \in \mathbb{R}^{d_h}$ and $b_y \in \mathbb{R}^{d_y}$ are bias vectors



<mark>실습</mark> `CustomRNN`을 완성하세요
- PyTorch 기본 모듈들(`nn.Linear`, `torch.tanh`)만을 사용하여 RNN을 구현하는 것이 목표입니다
- `torch.nn.RNN` 또는 `torch.nn.RNNCell`를 사용하지 <u>마세요</u>

In [ ]:
class CustomRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.W_xh = nn.Linear(input_dim, hidden_dim, bias = False)
        self.W_hh = nn.Linear(hidden_dim, hidden_dim, bias = True)
        self.W_hy = nn.Linear(hidden_dim, output_dim, bias = True)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        h_t = torch.zeros((batch_size, self.hidden_dim), device=x.device) # Initialize h_0 to zeros

        for t in range(seq_len):
            x_t = ...  # TODO
            h_t = ...  # TODO

        y_T = ...  # TODO

        return y_T

In [ ]:
model = CustomRNN(input_dim = 32, hidden_dim = 64, output_dim = 10)
X = torch.randn(4, 5, 32)  # (batch_size, seq_len, input_dim)
logits = model(X)
print("Logits shape:", logits.shape)

<mark>실습</mark> 앞서 정의한 `model = CustomRNN(input_dim = 32, hidden_dim = 64, output_dim = 10)`의 <b>학습 가능한 파라미터 수(learnable parameters)</b>를 직접 손으로 계산해 보세요.  
- 각 레이어의 **가중치(weight)**와 **편향(bias)**을 모두 포함해야 합니다.  
- 결과는 숫자 값으로 기입하거나, 숫자 계산식(예: 5*10)으로 입력해도 괜찮으나 **파이썬 변수를 사용하지 마세요**.

In [ ]:
num_params_customRNN = ... # TODO: Total number of learnable parameters in CustomRNN

In [ ]:
print(f"Total number of params : {num_params_customRNN}")

assert sum(p.numel() for p in model.parameters() if p.requires_grad) == num_params_customRNN, "❌ 계산한 파라미터 수가 실제 모델과 일치하지 않습니다."
print('\033[92mAll tests passed!')

<mark>실습</mark> 단어 분류 모델을 직접 학습(train)해보고 결과를 살펴보세요.

In [ ]:
def main():
    hidden_dim = 128
    batch_size = 1
    learning_rate = 0.005
    num_epochs = 2


    device = "cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
    print(f"Using device: {device}")

    names_dataset = NamesDataset(csv_path = "/datasets/NLP/names.csv")
    num_classes = names_dataset.num_classes

    train_dataset, val_dataset = torch.utils.data.random_split(names_dataset, [.85, .15], generator=torch.Generator().manual_seed(2025))
    print(f"train examples = {len(train_dataset)}, validation examples = {len(val_dataset)}")

    sample_weights = names_dataset.get_sample_weights()
    train_sample_weights = [sample_weights[i] for i in train_dataset.indices]

    sampler = torch.utils.data.sampler.WeightedRandomSampler(train_sample_weights, num_samples=len(train_sample_weights), replacement=True)
    train_dataloader = DataLoader(train_dataset, batch_size = batch_size, sampler=sampler)
    val_dataloader = DataLoader(val_dataset, batch_size = 1, shuffle=True)


    model = CustomRNN(input_dim = VOCAB_SIZE, hidden_dim = hidden_dim, output_dim = num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        train_one_epoch(model, device, train_dataloader, criterion, optimizer, epoch)
        evaluate_one_epoch(model, device, val_dataloader, criterion, epoch)

    draw_confusion_matrix(model, val_dataloader, names_dataset.class_names, device)
    return model

In [ ]:
model = main()

코드 구현이 잘 되었다면 별도의 하이퍼파라미터 튜닝(hyperparameter tuning)없이 `accuracy > 50%`를 달성하실 수 있습니다

새로운 이름에 대한 예측을 수행해보겠습니다.

In [ ]:
def predict(model, name, class_names, n_predictions, device):
    """
    Predict the label for a given name.
    
    Args:
        model (nn.Module): The RNN model.
        name (str): Input name string.
        class_names (list): List of class labels.
        n_predictions (int): Number of top predictions to return.
        device (torch.device): Computation device.
        
    Returns:
        list of tuples: Each tuple contains (probability, predicted_label).
    """
    model.eval()
    with torch.no_grad():
        name_tensor = one_hot_encode_string(name).to(device)  # [seq_len, VOCAB_SIZE]
        name_tensor = name_tensor.unsqueeze(0)  # [1, seq_len, VOCAB_SIZE]
        logits = model(name_tensor)  # [1, num_classes]

        probs = torch.softmax(logits, dim=-1)

        top_probs, top_indices = probs.topk(n_predictions, dim = 1)  # [1, k], [1, k]
        top_probs = top_probs.squeeze(0)
        top_indices = top_indices.squeeze(0)

        return [(float(p), class_names[i]) for p, i in zip(top_probs, top_indices)]

# Prediction examples.
example_names = ['Jackson', 'Satoshi', 'Choi']
print("\nPrediction Result:")
for name in example_names:
    preds = predict(model, name, names_dataset.class_names, n_predictions=3, device="cuda")
    print(f"\nInput Name: {name}")
    for prob, label in preds:
        print(f"  {label}: {prob:.2f}")

# Character-level language modeling
이번에는 RNN을 이용하여 문자 단위(character-level)의 언어 모델링(language modeling)을 수행해 보겠습니다.
## Language Model

언어 모델(Language Model)이란 특정 시점까지의 문자열들이 주어졌을 때, 그 다음에 올 문자열을 예측하는 모델입니다.

$$ P(w_t|w_1,w_2,...,w_{t-1})$$

 - 문자 단위(character-level) 토큰화: 이전 문자(character)들을 바탕으로 다음 문자를 예측
 - 단어 단위(word-level) 토큰화: 이전 단어(word)들을 바탕으로 다음 단어를 예측

## RNN 기반 문자 단위 언어 모델
RNN모델은 하나의 텍스트 파일을 입력으로 받아, 다음에 올 문자(next character)를 예측하는 작업을 학습하게 됩니다.

이렇게 학습된 RNN 언어 모델은 훈련 데이터와 유사한 문장 구조나 스타일을 모방한 <b>새로운 텍스트를 생성</b>하는데 사용될 수 있습니다.

이러한 언어 모델은 ChatGPT와 같은 대규모 언어 생성 모델의 기반 아이디어와 원리를 잘 보여줍니다.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm

## Shakespeare 데이터셋
먼저, 모델에 사용할 입력 데이터를 살펴보겠습니다.

이번 실습에서는 고전 문학 텍스트 중 하나인 <b>셰익스피어 작품의 일부</b>를 사용합니다.

In [ ]:
def load_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    return text

text = load_text("/datasets/NLP/tiny_shakespeare.txt")

print(text[:400])

## Build Vocabulary
입력 텍스트를 기반으로 Vocabulary를 생성합니다.

Vocabulary는 텍스트에 등장하는 모든 고유한 문자(character)들의 집합입니다.

In [ ]:
def build_vocab(text):
    vocab = sorted(set(text))
    char2idx = {c: i for i, c in enumerate(vocab)}
    idx2char = {i: c for i, c in enumerate(vocab)}
    return vocab, char2idx, idx2char

VOCAB, CHAR_TO_INDEX, INDEX_TO_CHAR = build_vocab(text)
VOCAB_SIZE = len(VOCAB)
print(f"Corpus: {len(text)} characters, Vocab size: {VOCAB_SIZE}")
print("Vocab: ", VOCAB)

Vocubulary를 이용하면 입력 text를 정수로 인코딩할 수 있습니다.

In [ ]:
integer_encoded_text = torch.tensor([CHAR_TO_INDEX[char] for char in text], dtype=torch.long)
print("Input text:", repr(text[:20]))
print("Encoded text:", integer_encoded_text[:20])

## Next character prediction
Language model을 학습하기 위해서는 주어진 문자 시퀀스를 입력받아, 그다음에 올 문자(next character)를 예측하는 작업을 수행할 수 있도록 데이터를 구성해야 합니다.

<center><img src="resources/char_language_model.jpg" style="width:800px;"></center>

위 그림은 `"Hello world!\n"`라는 문자열에 대한 예시를 보여줍니다.

* 각 시점(time-step) $t$에서 language model은 $P(x_t|x_1,x_2,...,x_{t-1})$를 예측하는 작업을 수행합니다
* $\mathbf{X} = (x_1, x_2, ..., x_T)$는 학습데이터에서 추출한 길이 $T$의 연속된 문자 시퀀스입니다.
* $\mathbf{y} = (y_1, y_2, ..., y_T)$는 예측할 타겟으로 $y_t = x_{t+1}$ 입니다.


<mark>실습</mark> `ShakespeareDataset`를 완성하세요
- 입력 시퀀스 $\mathbf{X}$는 문자를 one-hot encoding하여 리턴합니다
- 타겟 시퀀스 $\mathbf{y}$는 문자를 정수 인코딩된 상태 그대로 리턴합니다.

In [ ]:
class ShakespeareDataset(Dataset):
    def __init__(self, data, seq_length):
        """
        A custom dataset for training character-level language models.
        
        Args:
            data (Tensor): 1D tensor containing the text data, where each character is encoded as an integer.
            seq_length (int): The length of each input sequence (number of time steps)
        """
        self.data = data
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, idx):
        X = self.data[idx : idx + self.seq_length] # (T, )
        y = ... # TODO

        X_onehot = nn.functional.one_hot(X, num_classes = VOCAB_SIZE).float() # (T, |V|)
        return X_onehot, y

In [ ]:
seq_length = 100

dataset = ShakespeareDataset(integer_encoded_text, seq_length = seq_length)

for i in range(1):
    X, y = dataset[i]

    print(f"{i}-th example: ")
    print("X.shape:", X.shape)
    print("y.shape:", y.shape)
    print('- Input  (X):', repr(''.join([INDEX_TO_CHAR[char_idx.item()] for char_idx in X.argmax(dim=-1)])))
    print('- Target (y):', repr(''.join([INDEX_TO_CHAR[char_idx.item()] for char_idx in y])))
    print()


Expected output:

```python
X.shape: torch.Size([100, 65])
y.shape: torch.Size([100])
- Input  (X): 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
- Target (y): 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
```

In [ ]:
batch_size = 128

dataloader = DataLoader(dataset, batch_size = batch_size, shuffle=True, drop_last=True)

for X_batch, y_batch in dataloader:
    print(f"Input shape: {X_batch.shape}")
    print(f"Target shape: {y_batch.shape}")
    break 

- `X_batch`는 (batch_size, seq_length, VOCAB_SIZE)의 shape을 가집니다
- `y_batch`는 (batch_size, seq_length)의 shape을 가집니다.

---

<mark>실습</mark> `CharRNN` 모델을 완성하세요
- [`nn.RNN`](https://docs.pytorch.org/docs/stable/generated/torch.nn.RNN.html)를 활용하여 각 시점 $t$에서의 hidden state $\mathbf{h}_t$를 계산합니다.
  - `batch_first=True`로 설정하여 텐서의 shape이 `(batch_size, ...)` 형태가 되도록 합니다.
- `nn.RNN` 모듈의 출력값은 다음 두 가지로 구성됩니다 ([공식 문서](https://docs.pytorch.org/docs/stable/generated/torch.nn.RNN.html) 참고):
  - `output` : hidden features $\mathbf{h}_t$ from the last RNN layer for each $t$.
    - Shape : `(batch_size, seq_length, hidden_dim)`
  - `h_n`: 최종 시점 $T$에서의 hidden state $\mathbf{h}_T$.
    - Shape: `(num_layers, batch_size, hidden_dim)`
- 각 시점 $t$에 대하여 다음 문자를 예측합니다:
    $$\mathbf{\hat{y}}_t = \mathbf{W}_{hy}\mathbf{h}_t + b_y, \quad t=1,2,...,T$$
  - `self.fc`(`nn.Linear`)를 활용하세요


In [ ]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, hidden_dim, num_layers):
        super().__init__()
        self.rnn = nn.RNN(input_size = vocab_size, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden = None):
        """
        Args:
            x: input tensor of shape (batch_size, seq_len, vocab_size)
            hidden: previous hidden state of shape (num_layers, batch_size, hidden_dim)

        Returns:
            logits: tensor of shape (batch_size, seq_len, vocab_size)
            hidden: the last hidden state of shape (num_layers, batch_size, hidden_dim)
        """

        ##### YOUR CODE START #####


        ##### YOUR CODE END #####

        return logits, hidden

In [ ]:
model = CharRNN(vocab_size = VOCAB_SIZE, hidden_dim = 256, num_layers = 2)
X = torch.randint(0, 2, (batch_size, seq_length, VOCAB_SIZE)).float()  # (batch_size, seq_length, vocab_size)
logits, hidden = model(X)  # Example forward pass

print("X.shape:", X.shape)
print("logits.shape:", logits.shape)
print("hidden.shape:", hidden.shape)

<mark>실습</mark> 아래 코드를 이용하여 Language model을 학습해보세요.

In [ ]:
hidden_dim = 256
num_layers = 1

learning_rate = 1e-3
num_epochs = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = CharRNN(vocab_size = VOCAB_SIZE, hidden_dim = hidden_dim, num_layers = num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

model.train()
for epoch in range(num_epochs):
    dataloader_tqdm = tqdm(dataloader, desc=f'Training Epoch {epoch + 1}', total=len(dataloader))
    
    for X, y in dataloader_tqdm:
        X, y = X.to(device), y.to(device)
        
        logits, _ = model(X)

        # reshape for loss calculation: (batch*seq_len, vocab_size) vs (batch*seq_len)
        loss = criterion(logits.view(-1, VOCAB_SIZE), y.view(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        dataloader_tqdm.set_postfix({"loss": f"{loss.item():.4e}"})

    dataloader_tqdm.close()


## Autoregressive language generation

학습이 완료된 언어 모델로 새로운 텍스트를 생성할 때에는, 직전까지 예측된 문자들을 입력으로 사용하여 다음 문자를 예측하는 과정을 반복합니다.

이처럼 이전 출력 결과를 다시 입력으로 활용하는 구조를 Autoregressive(자기회귀) 모델이라고 부릅니다.

<center><img src="resources/autoregressive.jpg" style="width:800px;"></center>

- **Step 1**: Start with some character $x_1$ and zero hidden states $h_0$
- **Step 2**: Run one step of forward propagation to get $\hat{y}_1 \in [0,1]^{|V|}$.
- **Step 3**: 확률 분포 $\hat{y}_1$에 따라 다음 문자(character) $y_1$를 샘플링 합니다. 
- **Step 4**: Set $x_2 = y_1$ 
- Go to step 2 and repeat

In [ ]:
def sample_text(model, start_char, length=200, temperature=1.0):
    model.eval()

    input_idx = torch.tensor([[CHAR_TO_INDEX[start_char]]], device=device)
    hidden = None
    generated_text = start_char

    for _ in range(length):
        input_onehot = nn.functional.one_hot(input_idx, num_classes=VOCAB_SIZE).float()
        logits, hidden = model(input_onehot, hidden)

        logits = logits[:, -1, :] / temperature # (1, vocab_size)
        probs = torch.softmax(logits, dim=-1) # (1, vocab_size)

        next_idx = torch.multinomial(probs, num_samples=1)
        next_char = INDEX_TO_CHAR[next_idx.item()]
        generated_text += next_char

        input_idx = next_idx

    return generated_text

sample = sample_text(model, start_char='A', length=200)
print(f"Generated sentences: \n\n{sample}\n")

### Temperature sampling
모델의 출력값 logit $z_i\in \mathbb{R}^{|V|}$에 대하여, temperature-scaled softmax는 다음과 같이 정의산됩니다:

$$
p_i = \frac{\exp\!\bigl(\frac{z_i}{\tau}\bigr)}{\sum_{j}^{|V|} \exp\!\bigl(\frac{z_j}{\tau}\bigr)}
$$

- Temparature $\tau < 1$ 가 작을수록 분포가 뾰족해져 보수적인 샘플링을 수행 (high coherence)
- Temparature $\tau = 1$ : 기존 softmax와 동일한 분포에서 샘플링
- Temperature $\tau > 1$ 가 클수록 분포가 평탄해져 더 다양하고 창의적인 텍스트를 샘플링함 (high diversity)

In [ ]:
logits = torch.tensor([[1.0, 1.0, 3.0]])

print('Probabilities when temperature = 1:\t', nn.functional.softmax(logits, dim=1).numpy()[0])
print('Probabilities when temperature = 2:\t', nn.functional.softmax(logits / 2, dim=1).numpy()[0])
print('Probabilities when temperature = 0.5:\t', nn.functional.softmax(logits / 0.5 , dim=1).numpy()[0])
print('Probabilities when temperature = 0.01:\t', nn.functional.softmax(logits / 0.01, dim=1).numpy()[0])

In [ ]:
sample = sample_text(model, start_char='A', length=200, temperature=0.5)
print(f"Generated sentences: \n\n{sample}\n")